In [30]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/space-ship/sample_submission.csv
/kaggle/input/space-ship/train.csv
/kaggle/input/space-ship/test.csv
/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv
/kaggle/input/notebook4bc0bcc24a/x_train+oil.csv
/kaggle/input/notebook4bc0bcc24a/__results__.html
/kaggle/input/notebook4bc0bcc24a/__notebook__.ipynb
/kaggle/input/notebook4bc0bcc24a/__output__.json
/kaggle/input/notebook4bc0bcc24a/custom.css


In [44]:
## storing datasets with its csv name
for dataset in os.listdir('/kaggle/input/store-sales-time-series-forecasting'):
    split = dataset.split('.')
    globals()[split[0]] = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/'+dataset)
oil = oil.rename(columns = {oil.columns[1]: 'price'})
x_train , y_train = train.drop('sales' , axis = 1) , train[['sales']]

In [45]:
'''train.index = train.date
stores.index = stores.store_nbr
oil.index = oil.date
oil = oil.drop('date',axis = 1)
train = train.drop('date',axis = 1)
stores = stores.drop('store_nbr', axis = 1)
display(train.head(),stores.head(),oil.head())'''

"train.index = train.date\nstores.index = stores.store_nbr\noil.index = oil.date\noil = oil.drop('date',axis = 1)\ntrain = train.drop('date',axis = 1)\nstores = stores.drop('store_nbr', axis = 1)\ndisplay(train.head(),stores.head(),oil.head())"

In [46]:
##finding holidays in train dataset
lis = []
days = set(x_train.date.values)
for i in holidays_events.date:
    if i in days:
        lis.append(i)
lis[:5]

['2013-01-01', '2013-01-05', '2013-01-12', '2013-02-11', '2013-02-12']

In [47]:
for col in x_train.columns:
    print(f'the {col} column have type of   {x_train[col].dtypes}    and {x_train[col].nunique()} number of uniques value')
    

the id column have type of   int64    and 3000888 number of uniques value
the date column have type of   object    and 1684 number of uniques value
the store_nbr column have type of   int64    and 54 number of uniques value
the family column have type of   object    and 33 number of uniques value
the onpromotion column have type of   int64    and 362 number of uniques value


# preprocessing

In [48]:
x_train.index = train['id']
x_train.drop('id',axis =1,inplace = True , errors = 'ignore')
test.index = test['id']
x_test = test.drop('id',axis = 1)
x_train

,date,store_nbr,family,onpromotion
id,,,,
0,2013-01-01,1,AUTOMOTIVE,0
1,2013-01-01,1,BABY CARE,0
2,2013-01-01,1,BEAUTY,0
3,2013-01-01,1,BEVERAGES,0
4,2013-01-01,1,BOOKS,0
...,...,...,...,...
3000883,2017-08-15,9,POULTRY,0
3000884,2017-08-15,9,PREPARED FOODS,1
3000885,2017-08-15,9,PRODUCE,148


In [7]:
print('number of missing values in each column is:',train.isna().sum(),
      'so we dont need to have imputation',sep = '\n')


from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
numerical_cols = [col for col in x_train.columns if train[col].dtypes in ['int64','float64'] ]
categorical_cols = [col for col in x_train.columns if train[col].dtypes == 'object']
ordinal_cols = [col for col in categorical_cols if x_train[col].nunique() <100]
ordi = OrdinalEncoder()
train_ordi = ordi.fit_transform(x_train[categorical_cols])
transformer = ColumnTransformer([('pass','passthrough',x_train.columns.drop(categorical_cols)),
                               ('ordinal',OrdinalEncoder(),ordinal_cols)])
transferred_train = pd.DataFrame(transformer.fit_transform(train), columns = numerical_cols + ordinal_cols )
display(transferred_train)



number of missing values in each column is:
id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64
so we dont need to have imputation


,store_nbr,onpromotion,family
0,1.0,0.0,0.0
1,1.0,0.0,1.0
2,1.0,0.0,2.0
3,1.0,0.0,3.0
4,1.0,0.0,4.0
...,...,...,...
3000883,9.0,0.0,28.0
3000884,9.0,1.0,29.0
3000885,9.0,148.0,30.0
3000886,9.0,8.0,31.0


# modeling pipeline

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
model_linear = LinearRegression()
model_forest = RandomForestRegressor()
pipe1 = Pipeline(steps = [('preprocessing',transformer),
                        ('predictor', model_linear)])
pipe2 = Pipeline(steps = [('preprocessing',transformer),
                        ('predictor', model_forest)])
print('score of fitting on all dataset with linear regressor is :',pipe1.fit(x_train,y_train).score(x_train,y_train))
print('the mean accuracy for cross validation score with linear regressor model is :',cross_val_score(pipe1,x_train,y_train,cv =5).mean())

print('score of fitting on all dataset with random forest regressor is :',pipe2.fit(x_train,y_train).score(x_train,y_train))
print('the mean accuracy for cross validation score with random forest regressor model is :',cross_val_score(pipe2,x_train,y_train,cv =5).mean())

score of fitting on all dataset with linear regressor is : 0.19352675862082513
the mean accuracy for cross validation score with linear regressor model is : 0.13578534395034472


/opt/conda/lib/python3.10/site-packages/sklearn/pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


score of fitting on all dataset with random forest regressor is : 0.8498742777486561


/opt/conda/lib/python3.10/site-packages/sklearn/pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/opt/conda/lib/python3.10/site-packages/sklearn/pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/opt/conda/lib/python3.10/site-packages/sklearn/pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/opt/conda/lib/python3.10/site-packages/sklearn/pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change th

the mean accuracy for cross validation score with random forest regressor model is : 0.7671000045230771


as we saw, the score is very low so for improve it we should add more informations like oil price and ... to gain a better result

# concatenate datas

in the begining we want to add oil price to our dataset
so first we impute nan values in it

In [8]:
## impute nan values of oil price with mean of 50 days of its recents days prices
import math
oil_nan_index = [index for index in oil.index if math.isnan(oil['price'].iloc[index])]
print('befor imputation: ',oil.isna().sum(),sep = '\n')
for i in oil_nan_index:
    mean_limit = i//50
    oil.loc[i,'price'] = oil['price'][mean_limit*50:(mean_limit+1)*50].mean()

print('after imputation: ' , oil.isna().sum(),sep = '\n')

befor imputation: 
date      0
price    43
dtype: int64
after imputation: 
date     0
price    0
dtype: int64


concatenating oil price to each data row

In [9]:
## check that if we do it before and have our dateset do not repeat this time consuming stage

try:
    x_train_oil = pd.read_csv('/kaggle/input/notebook4bc0bcc24a/x_train+oil.csv',index_col = 'id')
except:
    for index , date in zip(x_train.index,x_train.date):
        if date in oil.date.values:
            x_train_oil.loc[index,'oil'] = oil[oil.date == date].price.values
        else:
            x_train_oil.loc[index,'oil'] = x_train.oil[index-1]

numerical_cols_oil = [col for col in x_train_oil.columns if x_train_oil[col].dtypes in ['int64','float64'] ]
categorical_cols_oil = [col for col in x_train_oil.columns if x_train_oil[col].dtypes == 'object']
ordinal_cols_oil = [col for col in categorical_cols_oil if x_train_oil[col].nunique() <100]
ordi = OrdinalEncoder()
train_ordi = ordi.fit_transform(x_train_oil[categorical_cols])
transformer2 = ColumnTransformer([('pass','passthrough',x_train_oil.columns.drop(categorical_cols_oil)),
                               ('ordinal',OrdinalEncoder(),ordinal_cols_oil)])
transferred_x_train_oil = pd.DataFrame(transformer2.fit_transform(x_train_oil), columns = numerical_cols_oil + ordinal_cols_oil )
transferred_x_train_oil

,store_nbr,onpromotion,oil,family
0,1.0,0.0,94.437447,0.0
1,1.0,0.0,94.437447,1.0
2,1.0,0.0,94.437447,2.0
3,1.0,0.0,94.437447,3.0
4,1.0,0.0,94.437447,4.0
...,...,...,...,...
3000883,9.0,0.0,47.570000,28.0
3000884,9.0,1.0,47.570000,29.0
3000885,9.0,148.0,47.570000,30.0
3000886,9.0,8.0,47.570000,31.0


In [11]:
model_linear2 = LinearRegression()
model_forest2 = RandomForestRegressor()
pipe_oil1 = Pipeline(steps = [('preprocessing',transformer2),
                        ('predictor', model_linear2)])
pipe_oil2 = Pipeline(steps = [('preprocessing',transformer2),
                        ('predictor', model_forest2)])
print('score of fitting on all dataset with linear regressor model is :',pipe_oil1.fit(x_train_oil,y_train).score(x_train_oil,y_train))
print('the mean accuracy for cross validation score with linear regressor model is :',cross_val_score(pipe_oil1,x_train_oil,y_train,cv =5).mean())

print('score of fitting on all dataset with random forest model is :',pipe_oil2.fit(x_train_oil,y_train).score(x_train_oil,y_train))
print('the mean accuracy for cross validation score with random forest model is :',cross_val_score(pipe_oil2,x_train_oil,y_train,cv =5).mean())

score of fitting on all dataset with linear regressor model is : 0.1936194154434422
the mean accuracy for cross validation score with linear regressor model is : 0.1338397475574738


/opt/conda/lib/python3.10/site-packages/sklearn/pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


score of fitting on all dataset with random forest model is : 0.967017250764213


/opt/conda/lib/python3.10/site-packages/sklearn/pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/opt/conda/lib/python3.10/site-packages/sklearn/pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/opt/conda/lib/python3.10/site-packages/sklearn/pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/opt/conda/lib/python3.10/site-packages/sklearn/pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change th

the mean accuracy for cross validation score with random forest model is : 0.6926676718328042


## XGBOOST regressor

In [16]:
import xgboost as xgb
xgb_regressor = xgb.XGBRegressor()
## without oil concatenation
pipe_xgb = Pipeline(steps = [('preprocessing',transformer),
                        ('predictor', xgb_regressor)])
pipe_oil_xgb = Pipeline(steps = [('preprocessing',transformer2),
                        ('predictor', xgb_regressor)])
print('score of fitting on all dataset with XGB regressor model is :' , pipe_xgb.fit(x_train,y_train).score(x_train,y_train))
score_xgb = cross_val_score(pipe_xgb , x_train , y_train , cv = 5)
print(f'the mean accuracy for cross validation score with XGB regressor model is ',score_xgb ,
      'its mean is:',score_xgb.mean())

print('score of fitting on all dataset with XGB regressor model is :' , pipe_oil_xgb.fit(x_train_oil,y_train).score(x_train_oil,y_train))
score_oil = cross_val_score(pipe_oil_xgb , x_train_oil , y_train , cv = 5)
print(f'the mean accuracy for cross validation score with linear regressor model is ',score_oil ,
      'its mean is:',score_oil.mean())


score of fitting on all dataset with XGB regressor model is : 0.8312108122123167
the mean accuracy for cross validation score with linear regressor model is  [0.64611268 0.76458467 0.81048587 0.79337289 0.85013636] its mean is: 0.7729384934388956
score of fitting on all dataset with XGB regressor model is : 0.8504609418539404
the mean accuracy for cross validation score with linear regressor model is  [0.64326986 0.72681906 0.76712382 0.81474153 0.83107616] its mean is: 0.7566060861073491


**best model was XGB regressor without oil price concatenation so we train model with it**


In [13]:
import xgboost as xgb
xgb_regressor = xgb.XGBRegressor()
## without oil concatenation
pipe_xgb = Pipeline(steps = [('preprocessing',transformer),
                        ('predictor', xgb_regressor)])

In [17]:
pipe_xgb.fit(x_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('pass', 'passthrough',
                                                  Index(['store_nbr', 'onpromotion'], dtype='object')),
                                                 ('ordinal', OrdinalEncoder(),
                                                  ['family'])])),
                ('predictor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              early_stopping_round...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [43]:
test.index = test['id']
x_test = test.drop('id',axis = 1)
x_test

KeyError: 'id'

In [52]:
sample_submission

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0
...,...,...
28507,3029395,0.0
28508,3029396,0.0
28509,3029397,0.0
28510,3029398,0.0


In [78]:
y_hat = pipe_xgb.predict(x_test)
submission = pd.DataFrame(y_hat,columns = ['sales'],index = test['id'])
submission.to_csv('submission_time_series.csv')
display(submission)

,sales
id,
3000888,-5.139878
3000889,-8.134998
3000890,-57.195251
3000891,1995.794067
3000892,-40.818596
...,...
3029395,549.580505
3029396,92.992889
3029397,1275.599243
